In [182]:
# Import librarys
import pandas as pd
import numpy as np
#from datetime import time

In [230]:
# Read in sheet
df = pd.read_excel('./data/2026-exam-timetable-excel-version.xlsx')

# Drop first column
df.drop(df.columns[0], axis=1, inplace=True)

# Rename columns
df.columns = ['A', 'B', 'C', 'D']

# Create a new column with the date
df['DATE'] = pd.to_datetime(df['A'], format='%d-%m-%Y',  errors='coerce')

# Drop rows with 'Course', 'Morning', 'Afternoon' in column A
df = df[df.A != 'Course']
df = df[df.A != 'Morning']
df = df[df.A != 'Afternoon']

# In Date column fill empty cells with the previous row value
df.DATE = df.DATE.ffill()

# Drop all rows that don’t have a certain at least 3 non-NaN values:
df = df.dropna(thresh=3)

# Remove suffixes from D column
df.D = df.D.str.removesuffix('* (approx)')
df.D = df.D.str.removesuffix('*')

# Split D column into two columns
df[['T1', 'T2']] = df['D'].str.split(' - ', n=1, expand=True)

# Create two columns with the date and time of the start and finish of exams
df['DT1'] = df.DATE.astype(str) + ' ' + df.T1.astype(str)
df['DT2'] = df.DATE.astype(str) + ' ' + df.T2.astype(str)
# Convert these two columns into datetime objects
df.DT1 = pd.to_datetime(df['DT1'])
df.DT2 = pd.to_datetime(df['DT2'])

# Create a new column with the Timedelta of the exam
df['L'] = df.DT2 - df.DT1 

# Add in columns with extra time
df['L%25'] = df['L'] * 1.25
df['L%50'] = df['L'] * 1.50

# Add in columns with end datetimes with extra time
df['DT2%25'] = df['DT1'] + df['L%25']
df['DT2%50'] = df['DT1'] + df['L%50']

# Add in columns with end datetimes with extra time
df['T2%25'] = df['DT2%25'].dt.strftime('%H:%M')
df['T2%50'] = df['DT2%50'].dt.strftime('%H:%M')

# Convert Timedelta into string (it doesn't seem to have a strftime function)
df['L1'] = df['L'].apply(lambda x: f'{x.components.hours:02d}:{x.components.minutes:02d}')
df['L1%25'] = df['L%25'].apply(lambda x: f'{x.components.hours:02d}:{x.components.minutes:02d}')
df['L1%50'] = df['L%50'].apply(lambda x: f'{x.components.hours:02d}:{x.components.minutes:02d}')

# Create new column with subject and level
df['A1'] = df['A'] + " - " + df['B']

df.head()

,A,B,C,D,DATE,T1,T2,DT1,DT2,L,L%25,L%50,DT2%25,DT2%50,T2%25,T2%50,L1,L1%25,L1%50,A1
4,Environmental Science,National 5,NaN,09:00 - 11:30,2026-04-22,09:00,11:30,2026-04-22 09:00:00,2026-04-22 11:30:00,0 days 02:30:00,0 days 03:07:30,0 days 03:45:00,2026-04-22 12:07:30,2026-04-22 12:45:00,12:07,12:45,02:30,03:07,03:45,Environmental Science - National 5
5,Spanish,Higher,Reading and Directed Writing,09:00 - 11:00,2026-04-22,09:00,11:00,2026-04-22 09:00:00,2026-04-22 11:00:00,0 days 02:00:00,0 days 02:30:00,0 days 03:00:00,2026-04-22 11:30:00,2026-04-22 12:00:00,11:30,12:00,02:00,02:30,03:00,Spanish - Higher
6,Spanish,Higher,Listening,11:30 - 12:00,2026-04-22,11:30,12:00,2026-04-22 11:30:00,2026-04-22 12:00:00,0 days 00:30:00,0 days 00:37:30,0 days 00:45:00,2026-04-22 12:07:30,2026-04-22 12:15:00,12:07,12:15,00:30,00:37,00:45,Spanish - Higher
7,Spanish,Advanced Higher,Reading and Translation,09:00 - 10:30,2026-04-22,09:00,10:30,2026-04-22 09:00:00,2026-04-22 10:30:00,0 days 01:30:00,0 days 01:52:30,0 days 02:15:00,2026-04-22 10:52:30,2026-04-22 11:15:00,10:52,11:15,01:30,01:52,02:15,Spanish - Advanced Higher
8,Spanish,Advanced Higher,Listening and Discursive Writing,11:00 - 12:20,2026-04-22,11:00,12:20,2026-04-22 11:00:00,2026-04-22 12:20:00,0 days 01:20:00,0 days 01:40:00,0 days 02:00:00,2026-04-22 12:40:00,2026-04-22 13:00:00,12:40,13:00,01:20,01:40,02:00,Spanish - Advanced Higher


In [235]:
# Create new df for final values
dfe = pd.DataFrame()

# Add in key columns
dfe['Subject'] = df.A
dfe['Level'] = df.B
dfe['Subject - Level'] = df.A1
dfe['Start Datetime'] = df.DT1
dfe['End Datetime'] = df.DT2
dfe['Start Date'] = df.DATE
dfe['Start Date'] = df.DATE
dfe['Start Time'] = df.T1
dfe['End Date'] = df.DATE
dfe['End Time'] = df.T2
dfe['Duration'] = df.L1
dfe['Description'] = df.C

# Add in extra time columns
dfe['Duration (+25%)'] = df['L1%25']
dfe['End Time (+25%)'] = df['T2%25']
dfe['End Datetime (+25%)'] = df['DT2%25']
dfe['Duration (+50%)'] = df['L1%50']
dfe['End Time (+50%)'] = df['T2%50']
dfe['End Datetime (+50%)'] = df['DT2%50']


dfe.head()

,Subject,Level,Subject - Level,Start Datetime,End Datetime,Start Date,Start Time,End Date,End Time,Duration,Description,Duration (+25%),End Time (+25%),End Datetime (+25%),Duration (+50%),End Time (+50%),End Datetime (+50%)
4,Environmental Science,National 5,Environmental Science - National 5,2026-04-22 09:00:00,2026-04-22 11:30:00,2026-04-22,09:00,2026-04-22,11:30,02:30,NaN,03:07,12:07,2026-04-22 12:07:30,03:45,12:45,2026-04-22 12:45:00
5,Spanish,Higher,Spanish - Higher,2026-04-22 09:00:00,2026-04-22 11:00:00,2026-04-22,09:00,2026-04-22,11:00,02:00,Reading and Directed Writing,02:30,11:30,2026-04-22 11:30:00,03:00,12:00,2026-04-22 12:00:00
6,Spanish,Higher,Spanish - Higher,2026-04-22 11:30:00,2026-04-22 12:00:00,2026-04-22,11:30,2026-04-22,12:00,00:30,Listening,00:37,12:07,2026-04-22 12:07:30,00:45,12:15,2026-04-22 12:15:00
7,Spanish,Advanced Higher,Spanish - Advanced Higher,2026-04-22 09:00:00,2026-04-22 10:30:00,2026-04-22,09:00,2026-04-22,10:30,01:30,Reading and Translation,01:52,10:52,2026-04-22 10:52:30,02:15,11:15,2026-04-22 11:15:00
8,Spanish,Advanced Higher,Spanish - Advanced Higher,2026-04-22 11:00:00,2026-04-22 12:20:00,2026-04-22,11:00,2026-04-22,12:20,01:20,Listening and Discursive Writing,01:40,12:40,2026-04-22 12:40:00,02:00,13:00,2026-04-22 13:00:00


In [236]:
dfe.to_csv('./csvs/sqa_exam_dates.csv', index = False)